# Get_Template: 
This script constructs a very simple `template` of a miyake event. The sinusoidal component of the production function is ignored resulting in a smooth curve. The template is constructed using the best fitting `model`-`dataset` pair  from the 774AD year. `emcee` is used to fit the parameters of the production function which is then used to generate the `.csv` file containing the `dc14` concentrations over just the event.

In [1]:
from os import getcwd
from ticktack.fitting import SingleFitter
from ticktack import load_presaved_model
from jax.numpy import array, mean, exp, sum, inf
from jax import jit
from csv import writer

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject


In [2]:
@jit
def super_gaussian(t, *args):
    const, start_time, duration, area = array(list(args)).reshape(-1)  # Loading parameters
    
    middle = start_time + duration / 2 # Middle of event 
    height = area / duration    # Amplitude of event
    gauss = height * exp(- ((t - middle) / (duration / 2)) ** 16)   # Gaussian pulse

    return const + gauss

@jit 
def log_likelihood(params, low_bounds, high_bounds):
    d14c = fitter.dc14(params)
    chi2 = -0.5 * sum(((fitter.d14c_data - d14c) / fitter.d14c_data_error) ** 2)

    chi2 += ((params[0] < low_bounds[0]) | (params[0] > high_bounds[0])) * -inf
    chi2 += ((params[1] < low_bounds[1]) | (params[1] > high_bounds[1])) * -inf
    chi2 += ((params[2] < low_bounds[2]) | (params[2] > high_bounds[2])) * -inf
    chi2 += ((params[3] < low_bounds[3]) | (params[3] > high_bounds[3])) * -inf
    return chi2

In [3]:
carbon_box_model = load_presaved_model(     # Generating the CarbobBoxModel using ticktack
    "Guttler14",                            # Selecting an arbitrary model 
    production_rate_units="atoms/cm^2/s"    # Units of the model
)

fitter = SingleFitter(carbon_box_model)   # Fitting a model 
fitter.prepare_function(model=super_gaussian) # Generating the simple sin model
fitter.load_data(f"{getcwd()}/datasets/775AD/NH/Miyake12_Cedar.csv")   

mcmc_samples = fitter.MarkovChainSampler(
    array([2.0, 775.0, 0.1, 0.1]),    # Initial guess of the parameters
    likelihood=log_likelihood,    
    args=(array([0.0, 770.0, 0.0, 0.0]), array([5.0, 780.0, 10.0, 10.0])) # Upper and lower bounds
)   

INFO[2022-01-17 07:28:43,747]: Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
INFO[2022-01-17 07:28:43,749]: Unable to initialize backend 'gpu': NOT_FOUND: Could not find registered platform with name: "cuda". Available platform names are: Host Interpreter
INFO[2022-01-17 07:28:43,750]: Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
WARNING[2022-01-17 07:28:43,751]: No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Running burn-in...


100%|██████████| 500/500 [00:41<00:00, 11.99it/s]


Running production...


100%|██████████| 1000/1000 [01:28<00:00, 11.27it/s]


In [4]:
event_parameters = mean(mcmc_samples, axis=0)   # The mcmc fitted parameters